<a href="https://colab.research.google.com/github/npv2k1/colab-starter/blob/main/colab_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect google drive to google colab
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Install openssh server
!apt update ; apt install openssh-server
!echo 'PermitRootLogin yes' >> /etc/ssh/sshd_config
!mkdir /root/.ssh
!echo 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCf9v5Zr+tbkFhAYaLiih5CkNjs8YjUNuy46AKUHy/RcLsxkJNoL3XJ1f+a5RpdmALJocJkQFGmf5eq39ZaY/vgezn7EuXiI+czb7eK+5Nq4Wj9jtUujuzVszZYN/dX5dmDR62/DN8ZyZbAwrimpPKnaqOywfIrP6mVpbuo855f75nT4sT1t0A8pJ4+Qi1TNOEmWngz2MlbVcSZW5nnTa+PL2l16uklX5QY50uD1NxgWcs0rae52tbBqm1vrpNihxC1jQ02xL6XmBYRhJQi67QAxmUE6vKseG3MHGESbrkjhbr+0L59nheSn26kCPuO23kjGRKtHw5Qc+2IR+E73foMQ61qCVN5WO8OyjxujJuLF4mEmu8u61rzttiaHmPn6RYYYtGXTFg7HmQcBLwDqppUejH4ku6xN/aSZFeJNMFs7wJcIJ1GvtC9bqtmhrbe1tdZNaGDjC+KHgE8YyN3fo5JYqDsIpXrX/FPKRsJtwYsce67r6D6Rxi2OS9fe6iyzVc= nguyen@DESKTOP-Nguyen' >/root/.ssh/authorized_keys
!service ssh start

In [ ]:
# Install tailscale VPN and save state to google drive
!curl -fsSL https://tailscale.com/install.sh | sh
!nohup sudo tailscaled --tun=userspace-networking --socks5-server=localhost:1055 --state=/content/gdrive/MyDrive/tailscaled.state --socket=/var/run/tailscale/tailscaled.sock --port 41641 &
!until tailscale up; do sleep 1; done

In [ ]:
!tailscale ip

In [ ]:
!nvidia-smi

# Công cụ

In [2]:
# ============================= FORM ============================= #
#@markdown Download file
#@markdown
url = "https://az764295.vo.msecnd.net/stable/f1b07bd25dfad64b0167beb15359ae573aecd2cc/code-stable-x64-1696981541.tar.gz" #@param {type:"string"}
download_dir = "/content/downloads/" #@param {type:"string"}
# ============================= FORM ============================= #

import requests
import os
from tqdm import tqdm

# Ensure the download directory exists
os.makedirs(download_dir, exist_ok=True)

# Extract the file name from the URL
file_name = os.path.join(download_dir, url.split("/")[-1])

# Send an HTTP GET request to the URL
response = requests.get(url, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Get the total file size in bytes from the response headers
    total_size = int(response.headers.get('content-length', 0))

    # Initialize the progress bar
    progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

    # Download and save the content of the response with a progress bar
    with open(file_name, 'wb') as file:
        for data in response.iter_content(chunk_size=1024):
            progress_bar.update(len(data))
            file.write(data)

    progress_bar.close()

    print(f"File downloaded to {file_name}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


100%|██████████| 136M/136M [00:01<00:00, 77.5MB/s]

File downloaded to /content/downloads/code-stable-x64-1696981541.tar.gz


In [3]:
# ============================= FORM ============================= #
#@markdown Giải nén file
#@markdown
archive_path = "/content/downloads/code-stable-x64-1696981541.tar.gz" #@param {type:"string"}
extract_dir = "/content/downloads/" #@param {type:"string"}
# ============================= FORM ============================= #


import os
import subprocess
from tqdm import tqdm

# Ensure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract the archive using the system tar command
command = ["tar", "xzvf", archive_path, "-C", extract_dir]

# Get the total size of the archive
total_size = os.path.getsize(archive_path)

with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as progress_bar:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    for line in process.stdout:
        progress_bar.update(len(line.encode('utf-8')))

    process.communicate()  # Wait for the process to finish

    if process.returncode == 0:
        print(f"Extraction complete to {extract_dir}")
    else:
        print(f"Extraction failed. Return code: {process.returncode}")


  0%|          | 105k/130M [00:06<2:10:05, 17.4kB/s]

Extraction complete to /content/downloads/


In [7]:
# ============================= FORM ============================= #
#@markdown Nén thư mục
#@markdown
folder_to_archive = "/content/downloads/VSCode-linux-x64" #@param {type:"string"}
archive_path = "/content/downloads/archive.tar.gz" #@param {type:"string"}
# ============================= FORM ============================= #


import shutil
import os
from tqdm import tqdm


# Create the archive using the system tar command
command = ["tar", "czvf", archive_path, "-C", os.path.dirname(folder_to_archive), os.path.basename(folder_to_archive)]

# Get the total size of the folder
def get_folder_size(folder):
    total_size = 0
    for dirpath, _, filenames in os.walk(folder):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

total_size = get_folder_size(folder_to_archive)

# Display a progress bar while creating the archive
with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as progress_bar:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    for line in process.stdout:
        progress_bar.update(len(line.encode('utf-8')))

    process.communicate()  # Wait for the process to finish

    if process.returncode == 0:
        print(f"Archive created at: {archive_path}")
    else:
        print(f"Archive creation failed. Return code: {process.returncode}")

  0%|          | 105k/366M [00:24<24:43:56, 4.31kB/s]

Archive created at: /content/downloads/archive.tar.gz


In [ ]:
# ============================= FORM ============================= #
#@markdown Sao chép
#@markdown
source = "/content/downloads/VSCode-linux-x64" #@param {type:"string"}
destination = "/content/vscode" #@param {type:"string"}
# ============================= FORM ============================= #


import shutil
import os
from tqdm import tqdm

# Copy the source to the destination
shutil.copy(source, destination)


# Git

In [9]:
# ============================= FORM ============================= #
#@markdown Clone repo
#@markdown
repo_url = "https://github.com/pnstack/colab-starter.git" #@param {type:"string"}
clone_directory = "/content/source" #@param {type:"string"}

# ============================= FORM ============================= #


import subprocess


# Run the Git clone command
try:
    subprocess.run(['git', 'clone', repo_url, clone_directory], check=True)
    print(f"Repository cloned to {clone_directory}")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

Repository cloned to /content/source
